In [29]:
import  tensorflow as tf
from    tensorflow import keras
import tensorflow.keras.backend as K
from    tensorflow.keras import layers, models, Sequential, backend
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate, Lambda, Input, ZeroPadding2D, AveragePooling2D, DepthwiseConv2D, Reshape

def relu6(x):
    return K.relu(x, max_value=6)

# 保證為8的倍數
def make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v+divisor/2)//divisor*divisor) #//向下取整，除
    if new_v<0.9*v:
        new_v +=divisor
    return new_v

def pad_size(inputs, kernel_size):

    input_size = inputs.shape[1:3]

    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)

    if input_size[0] is None:
        adjust = (1,1)

    else:
        adjust = (1- input_size[0]%2, 1-input_size[1]%2)
    
    correct = (kernel_size[0]//2, kernel_size[1]//2)

    return ((correct[0] - adjust[0], correct[0]),
            (correct[1] - adjust[1], correct[1]))

def conv_block (x, nb_filter, kernel=(1,1), stride=(1,1), name=None):

    x = Conv2D(nb_filter, kernel, strides=stride, padding='same', use_bias=False, name=name+'_expand')(x)
    x = BatchNormalization(axis=3, name=name+'_expand_BN')(x)
    x = Activation(relu6, name=name+'_expand_relu')(x)

    return x


def depthwise_res_block(x, nb_filter, kernel, stride, t, alpha, resdiual=False, name=None):

    input_tensor=x
    exp_channels= x.shape[-1]*t  #扩展维度
    alpha_channels = int(nb_filter*alpha)     #压缩维度

    x = conv_block(x, exp_channels, (1,1), (1,1), name=name)

    if stride[0]==2:
        x = ZeroPadding2D(padding=pad_size(x, 3), name=name+'_pad')(x)

    x = DepthwiseConv2D(kernel, padding='same' if stride[0]==1 else 'valid', strides=stride, depth_multiplier=1, use_bias=False, name=name+'_depthwise')(x)

    x = BatchNormalization(axis=3, name=name+'_depthwise_BN')(x)
    x = Activation(relu6, name=name+'_depthwise_relu')(x)

    x = Conv2D(alpha_channels, (1,1), padding='same', use_bias=False, strides=(1,1), name=name+'_project')(x)
    x = BatchNormalization(axis=3, name=name+'_project_BN')(x)

    if resdiual:
        x = layers.add([x, input_tensor], name=name+'_add')

    return x

def MovblieNetV2 (alpha=1., dropout=0):

    img_input = Input(shape=(540, 960, 3))

    first_filter = make_divisible(32*alpha, 8)
    
    x = ZeroPadding2D(padding=pad_size(img_input, 3), name='Conv1_pad')(img_input)
    x = Conv2D(first_filter, (3,3), strides=(2,2), padding='valid', use_bias=False, name='Conv1')(x)
    x = BatchNormalization(axis=3, name='bn_Conv1')(x)
    x = Activation(relu6, name='Conv1_relu')(x)

    x = DepthwiseConv2D((3,3), padding='same', strides=(1,1), depth_multiplier=1, use_bias=False, name='expanded_conv_depthwise')(x)
    x = BatchNormalization(axis=3, name='expanded_conv_depthwise_BN')(x)
    x = Activation(relu6, name='expanded_conv_depthwise_relu')(x)

    x = Conv2D(16, (1,1), padding='same', use_bias=False, strides=(1,1), name='expanded_conv_project')(x)
    x = BatchNormalization(axis=3, name='expanded_conv_project_BN')(x)

    x = depthwise_res_block(x, 24, (3,3), (2,2), 6, alpha, resdiual=False, name='block_1') 

    x = depthwise_res_block(x, 24, (3,3), (1,1), 6, alpha, resdiual=True, name='block_2') 

    x = depthwise_res_block(x, 32, (3,3), (2,2), 6, alpha, resdiual=False, name='block_3')

    x = depthwise_res_block(x, 32, (3,3), (1,1), 6, alpha, resdiual=True, name='block_4')

    x = depthwise_res_block(x, 32, (3,3), (1,1), 6, alpha, resdiual=True, name='block_5')

    x = depthwise_res_block(x, 64, (3,3), (1,1), 6, alpha, resdiual=False, name='block_6')
    
    x = depthwise_res_block(x, 64, (3,3), (1,1), 6, alpha, resdiual=True, name='block_7')

    x = depthwise_res_block(x, 64, (3,3), (1,1), 6, alpha, resdiual=True, name='block_8')

    x = depthwise_res_block(x, 64, (3,3), (1,1), 6, alpha, resdiual=True, name='block_9')

    x = depthwise_res_block(x, 96, (3,3), (1,1), 6, alpha, resdiual=False, name='block_10')

    x = depthwise_res_block(x, 96, (3,3), (1,1), 6, alpha, resdiual=True, name='block_11')

    x = depthwise_res_block(x, 96, (3,3), (1,1), 6, alpha, resdiual=True, name='block_12')

    model = models.Model(img_input, x, name='MobileNetV2')
#     print(model.summary())
    return model

# x = MovblieNetV2()




In [32]:
# 假設你已經有 img_input 作為輸入
img_input = Input(shape=(540, 960, 3))
# 使用MobileNetV2
mobilenetv2 = MovblieNetV2()(img_input) 
x = Conv2D(filters=96, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2,groups=96,activation='relu',use_bias=False)(mobilenetv2)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=96,activation='relu',use_bias=False)(x)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu',use_bias=False)(x)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
# 最後輸出density map
density_map = Conv2D(filters=1, kernel_size=1, name='density_map')(x)
# 建立模型
model = Model(img_input, density_map,name = 'CSRnet')
print(model.summary())


Model: "CSRnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 540, 960, 3)]     0         
_________________________________________________________________
MobileNetV2 (Functional)     (None, 68, 120, 96)       558656    
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 68, 120, 96)       864       
_________________________________________________________________
batch_normalization_166 (Bat (None, 68, 120, 96)       384       
_________________________________________________________________
conv2d_130 (Conv2D)          (None, 68, 120, 192)      1728      
_________________________________________________________________
batch_normalization_167 (Bat (None, 68, 120, 192)      768       
_________________________________________________________________
conv2d_131 (Conv2D)          (None, 68, 120, 192)      1728 

In [17]:
# import  tensorflow as tf
# from    tensorflow import keras
# import tensorflow.keras.backend as K
# from    tensorflow.keras import layers, models, Sequential, backend,Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
# from tensorflow.keras.layers import Concatenate, Lambda, Input, ZeroPadding2D, AveragePooling2D, DepthwiseConv2D, Reshape,Add


# def _make_divisible(v, divisor, min_value=None):
#     if min_value is None:
#         min_value = divisor
#     new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
#     # Make sure that round down does not go down by more than 10%.
#     if new_v < 0.9 * v:
#         new_v += divisor
#     return new_v


# def relu6(x):
#     """Relu 6
#     """
#     return K.relu(x, max_value=6.0)


# def _conv_block(inputs, filters, kernel, strides):
#     """Convolution Block
#     This function defines a 2D convolution operation with BN and relu6.

#     # Arguments
#         inputs: Tensor, input tensor of conv layer.
#         filters: Integer, the dimensionality of the output space.
#         kernel: An integer or tuple/list of 2 integers, specifying the
#             width and height of the 2D convolution window.
#         strides: An integer or tuple/list of 2 integers,
#             specifying the strides of the convolution along the width and height.
#             Can be a single integer to specify the same value for
#             all spatial dimensions.

#     # Returns
#         Output tensor.
#     """

#     channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

#     x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
#     x = BatchNormalization(axis=channel_axis)(x)
#     return Activation(relu6)(x)


# def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
#     """Bottleneck
#     This function defines a basic bottleneck structure.

#     # Arguments
#         inputs: Tensor, input tensor of conv layer.
#         filters: Integer, the dimensionality of the output space.
#         kernel: An integer or tuple/list of 2 integers, specifying the
#             width and height of the 2D convolution window.
#         t: Integer, expansion factor.
#             t is always applied to the input size.
#         s: An integer or tuple/list of 2 integers,specifying the strides
#             of the convolution along the width and height.Can be a single
#             integer to specify the same value for all spatial dimensions.
#         alpha: Integer, width multiplier.
#         r: Boolean, Whether to use the residuals.

#     # Returns
#         Output tensor.
#     """

#     channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
#     # Depth
#     tchannel = K.int_shape(inputs)[channel_axis] * t
#     # Width
#     cchannel = int(filters * alpha)

#     x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

#     x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation(relu6)(x)

#     x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
#     x = BatchNormalization(axis=channel_axis)(x)

#     if r:
#         x = Add()([x, inputs])

#     return x


# def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
#     """Inverted Residual Block
#     This function defines a sequence of 1 or more identical layers.

#     # Arguments
#         inputs: Tensor, input tensor of conv layer.
#         filters: Integer, the dimensionality of the output space.
#         kernel: An integer or tuple/list of 2 integers, specifying the
#             width and height of the 2D convolution window.
#         t: Integer, expansion factor.
#             t is always applied to the input size.
#         alpha: Integer, width multiplier.
#         s: An integer or tuple/list of 2 integers,specifying the strides
#             of the convolution along the width and height.Can be a single
#             integer to specify the same value for all spatial dimensions.
#         n: Integer, layer repeat times.

#     # Returns
#         Output tensor.
#     """

#     x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

#     for i in range(1, n):
#         x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

#     return x


# def MobileNetv2(alpha=1.0):
#     """MobileNetv2
#     This function defines a MobileNetv2 architectures.

#     # Arguments
#         input_shape: An integer or tuple/list of 3 integers, shape
#             of input tensor.
#         k: Integer, number of classes.
#         alpha: Integer, width multiplier, better in [0.35, 0.50, 0.75, 1.0, 1.3, 1.4].

#     # Returns
#         MobileNetv2 model.
#     """
#     inputs = Input(shape=(1920,1080,3))

#     first_filters = _make_divisible(32 * alpha, 8)
#     x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

#     x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=alpha, strides=1, n=1)
#     x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=alpha, strides=2, n=2)
#     x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=alpha, strides=2, n=3)
#     x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=alpha, strides=2, n=4)
#     x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=alpha, strides=1, n=3)
# #     x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
# #     x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)


#     model = Model(inputs, x)
#     # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)
#     print(model.summary())

#     return model

# x = MobileNetv2()

In [20]:

"""MobileNet v2 models for Keras.
# Reference
- [Inverted Residuals and Linear Bottlenecks Mobile Networks for
   Classification, Detection and Segmentation]
   (https://arxiv.org/abs/1801.04381)
"""
 
 
import  tensorflow as tf
from    tensorflow import keras
import tensorflow.keras.backend as K
from    tensorflow.keras import layers, models, Sequential, backend
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate, Lambda, Input, ZeroPadding2D, AveragePooling2D, DepthwiseConv2D, Reshape
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
# from keras import backend as K
 
 
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v
 
 
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)
 
 
def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """
 
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
 
    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)
 
 
def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        alpha: Integer, width multiplier.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """
 
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    # Depth
    tchannel = K.int_shape(inputs)[channel_axis] * t
    # Width
    cchannel = int(filters * alpha)
 
    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))
 
    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)
 
    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
 
    if r:
        x = Add()([x, inputs])
 
    return x
 
 
def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        alpha: Integer, width multiplier.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """
 
    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)
 
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)
 
    return x
 
 
def MobileNetv2():
    """MobileNetv2
    This function defines a MobileNetv2 architectures.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        alpha: Integer, width multiplier, better in [0.35, 0.50, 0.75, 1.0, 1.3, 1.4].
    # Returns
        MobileNetv2 model.
    """
    inputs = Input(shape=(540, 960, 3))
    alpha = 1.0
    first_filters = _make_divisible(32 * alpha, 8)
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))
 
    x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=1.0, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=1.0, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=1.0, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=1.0, strides=1, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=1.0, strides=1, n=3)
#     x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
#     x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)
 
    model = Model(inputs, x)
#     model.summary()
    return model
 
 
# model = MobileNetv2((960,540, 3), 1.0)
# print(model.summary())


In [19]:
# 假設你已經有 img_input 作為輸入
img_input = Input(shape=(540, 960, 3))
# 使用MobileNetV2
mobilenetv2 = MobileNetv2()(img_input) 
x = Conv2D(filters=96, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2,groups=96,activation='relu',use_bias=False)(mobilenetv2)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=96,activation='relu',use_bias=False)(x)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu',use_bias=False)(x)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=2, groups=192,activation='relu')(x)
# 最後輸出density map
density_map = Conv2D(filters=1, kernel_size=1, name='density_map')(x)
# 建立模型
model = Model(img_input, density_map,name = 'CSRnet')
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 540, 960, 3) 0                                            
__________________________________________________________________________________________________
conv2d_176 (Conv2D)             (None, 270, 480, 32) 896         input_15[0][0]                   
__________________________________________________________________________________________________
batch_normalization_251 (BatchN (None, 270, 480, 32) 128         conv2d_176[0][0]                 
__________________________________________________________________________________________________
activation_168 (Activation)     (None, 270, 480, 32) 0           batch_normalization_251[0][0]    
____________________________________________________________________________________________

Model: "CSRnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 540, 960, 3)]     0         
_________________________________________________________________
model_4 (Functional)         (None, 68, 120, 96)       567904    
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 68, 120, 96)       864       
_________________________________________________________________
batch_normalization_291 (Bat (None, 68, 120, 96)       384       
_________________________________________________________________
conv2d_204 (Conv2D)          (None, 68, 120, 192)      1728      
_________________________________________________________________
batch_normalization_292 (Bat (None, 68, 120, 192)      768       
_________________________________________________________________
conv2d_205 (Conv2D)          (None, 68, 120, 192)      1728 

In [53]:
import  tensorflow as tf

def conv_block (x, filters, kernel=(1,1), stride=(1,1)):

    x = tf.keras.layers.Conv2D(filters, kernel, strides=stride, padding='same',use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.nn.relu6)(x)

    return x

def depthwise_res_block(x, filters, kernel, stride, t, resdiual=False):

    input_tensor = x
    exp_channels = x.shape[-1]*t  

    x = conv_block(x, exp_channels, (1,1), (1,1))

    x = tf.keras.layers.DepthwiseConv2D(kernel, padding='same', strides=stride,use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.nn.relu6)(x)

    x = tf.keras.layers.Conv2D(filters, (1,1), padding='same', strides=(1,1),use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    if resdiual:
        x = tf.keras.layers.add([x, input_tensor])

    return x

def inverted_residual_layers(x, filters, stride, t, n):

    x = depthwise_res_block(x, filters, (3,3), stride, t, False)

    for i in range(1, n):
        x = depthwise_res_block(x, filters, (3,3), (1,1), t, True)
    
    return x
def MovblieNetV2 ():

    img_input = tf.keras.layers.Input(shape=(540, 960, 3))

    x = conv_block(img_input, 32, (3,3), (2,2))

    x = tf.keras.layers.DepthwiseConv2D((3,3), padding='same', strides=(1,1),use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.nn.relu6)(x)
    
    x = Conv2D(16, (1,1), padding='same', use_bias=False, strides=(1,1))(x)
    x = BatchNormalization()(x)
#     x = inverted_residual_layers(x, 16, (1,1), 1, 1)
    x = inverted_residual_layers(x, 24, (2,2), 6, 2)
    x = inverted_residual_layers(x, 32, (2,2), 6, 3)
    x = inverted_residual_layers(x, 64, (1,1), 6, 4)
    x = inverted_residual_layers(x, 96, (1,1), 6, 3)
#     x = inverted_residual_layers(x, 160, (2,2), 6, 3)
#     x = inverted_residual_layers(x, 320, (1,1), 6, 1)
    model = tf.keras.Model(img_input, x)

    return model


In [54]:
model = MovblieNetV2()
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 540, 960, 3) 0                                            
__________________________________________________________________________________________________
conv2d_510 (Conv2D)             (None, 270, 480, 32) 864         input_31[0][0]                   
__________________________________________________________________________________________________
batch_normalization_749 (BatchN (None, 270, 480, 32) 128         conv2d_510[0][0]                 
__________________________________________________________________________________________________
activation_505 (Activation)     (None, 270, 480, 32) 0           batch_normalization_749[0][0]    
___________________________________________________________________________________________

In [50]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
def MovblieNetV():
    img_input = Input(shape=(540, 960, 3))

    mobilenetv2 = MobileNetV2(include_top=False, weights='imagenet', input_tensor=img_input)
    # mobilenetv2 = MobileNetV2(include_top=False, input_tensor=img_input)
    # x = mobilenetv2.layers[115].output
    x = mobilenetv2.get_layer('block_12_add').output

    model = Model(img_input, x)
    return model

m = MovblieNetV()
m.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 540, 960, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 270, 480, 32) 864         input_29[0][0]                   
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 270, 480, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 270, 480, 32) 0           bn_Conv1[0][0]                   
___________________________________________________________________________________________